#### **Density Profile Comparisons**:

Notebook for matching halos between the Illustris_TNG and Illustris_TNG_Dark catalogs

In [7]:
import numpy as np
import illustris_python as il
import matplotlib.pyplot as plt 
from matplotlib.patches import Circle
import seaborn as sns
import h5py
from astropy.cosmology import Planck15 as cosmo
import sys


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

sns.set(style = "ticks")
pi = np.pi

%config IPCompleter.greedy = True
%config InlineBackend.figure_format ='retina'



# basePath = '../Splashback-Research/TNG-100_3/outputs/'
# basePath_dark = '../Splashback-Research/TNG-100_3_Dark/outputs/'

basePath = '../Splashback-Research/TNG-300_3/outputs/'
basePath_dark = '../Splashback-Research/TNG-300_3_Dark/outputs/'

snap_dir_g = basePath+'snapdir_066/snap_066.0.hdf5'
snap_dir_dark = basePath_dark+'snapdir_066/snap_066.0.hdf5'

snapshot_ind = 66

# Load header file and halo group catalog
header = il.groupcat.loadHeader(basePath, snapshot_ind)
header_dark = il.groupcat.loadHeader(basePath_dark, snapshot_ind)

# Obtain basic cosmological info and obtain conversion factors
redshift = header['Redshift']
H = cosmo.H(redshift)
H0 = cosmo.H(0)
h = H0.value/100

# Define box boundary vector in Mpc (for period bcs)
boxsize = header['BoxSize'] # kpc/h
box_bounds = boxsize/(1000*h)*np.ones(3)


# Get all profiles with halo mass between 10^13-10^14 solar mass
halo_masses= il.groupcat.loadHalos(basePath, snapshot_ind, fields=['GroupMass'])*10**10/h
halo_masses_dark = il.groupcat.loadHalos(basePath_dark, snapshot_ind, fields=['GroupMass'])*10**10/h

halo_indices = np.where((halo_masses >= 10**13) & 
                        (halo_masses <= 10**14))[0]
halo_indices_dark = np.where((halo_masses_dark >= 10**13) & 
                             (halo_masses_dark <= 10**14))[0]




In [ ]:
def progress_bar(cur_val, final_val):
    """ 
    Function to keep track of progress during computations by displaying
    a progress bar

    Parameters:
    cur_val (int/float): current iteration/value calculation is on
    final_val (int/float): final iteration/value that calculation will take
    """

    bar_length = 20
    percent = float(cur_val) / final_val
    arrow = '-' * int(round(percent * bar_length)-1) + '>'
    spaces = ' ' * (bar_length - len(arrow))

    sys.stdout.write("\rProgress: [{0}]"
                     " {1}%".format(arrow + spaces, int(round(percent * 100))))
    sys.stdout.flush()

            
# Compute associated indices for full snapshot halos 
similar_ind = np.empty(len(halo_indices), dtype=tuple)

def find_most_similar_ind(halo_ind, basePath_1, halo_indices_2, basePath_2):

    """
    Parameters:
    ----------
    halo_ind: int
        x, y, and z positions of particles

    basePath_1:
        string containing path of catalog which halo_ind is from

    halo_indices_2:
        All possible candidate halos in basePath_2
    
    basePath_2:
        string containing path of catalog which halo_indices_2 are from

    Returns:
    -------
    halo_ind_2: 
    """
    
    particle_ids_1 = il.snapshot.loadHalo(basePath_1, snapshot_ind, halo_ind,'dm')['ParticleIDs']
    
    inter_max = 0
    halo_ind_2 = None
    
    for i in range(len(halo_indices_2)):
        cur_ind = halo_indices_2[i]
        particle_ids_2 = il.snapshot.loadHalo(basePath_1, snapshot_ind, cur_ind,'dm')['ParticleIDs']
        
        inter_cur = len(np.intersect1d(particle_ids, particle_ids_dark))
        
        if inter_cur > inter_max:
            inter_max = inter_cur
            halo_ind_2 = cur_ind
    

for i in range(len(halo_indices)):
    
    progress_bar(i, len(halo_indices))
    halo_ind = halo_indices[i]
    particle_ids = il.snapshot.loadHalo(basePath, snapshot_ind, halo_ind,'dm')['ParticleIDs']
    
    inter_max = 0
    
    for j in range(len(halo_indices_dark)):
        halo_ind_dark = halo_indices_dark[j]
        particle_ids_dark =  il.snapshot.loadHalo(basePath_dark, snapshot_ind, halo_ind_dark,'dm')['ParticleIDs']

        inter_cur = len(np.intersect1d(particle_ids, particle_ids_dark))
        
        if inter_cur > inter_max:
            inter_max = inter_cur
            similar_ind[i] = (halo_ind, halo_ind_dark)
            

Progress: [->                  ] 11%

In [152]:
# Compute associated indices for dark snapshot halos 
similar_ind_dark = np.empty(len(halo_indices_dark), dtype=tuple)

for i in range(len(halo_indices_dark)):
    
    progress_bar(i, len(halo_indices_dark))
    halo_ind_dark = halo_indices_dark[i]
    particle_ids_dark = il.snapshot.loadHalo(basePath_dark, snapshot_ind, halo_ind_dark,'dm')['ParticleIDs']
    
    inter_max = 0
    
    for j in range(len(halo_indices)):
        halo_ind = halo_indices[j]
        particle_ids =  il.snapshot.loadHalo(basePath, snapshot_ind, halo_ind,'dm')['ParticleIDs']

        inter_cur = len(np.intersect1d(particle_ids, particle_ids_dark))
        
        if inter_cur > inter_max:
            inter_max = inter_cur
            similar_ind_dark[i] = (halo_ind_dark, halo_ind)
            

Progress: [----------------->  ] 90%

In [160]:
good_ind = np.array([])
count = 0

# Remove values with no association
similar_ind = similar_ind[np.where(similar_ind != None)].tolist()
similar_ind_dark = similar_ind_dark[np.where(similar_ind_dark != None)].tolist()

for ind in similar_ind:

    if not (tuple(np.flip(ind)) in similar_ind_dark):
        np.delete(similar_ind, ind)
        

In [3]:
def find_most_similar_ind(halo_ind, basePath_1, halo_indices_2, basePath_2):

    """
    Parameters:
    ----------
    halo_ind: int
        x, y, and z positions of particles

    basePath_1:
        string containing path of catalog which halo_ind is from

    halo_indices_2:
        All possible candidate halos in basePath_2
    
    basePath_2:
        string containing path of catalog which halo_indices_2 are from

    Returns:
    -------
    halo_ind_2: 
    """
    
    particle_ids_1 = il.snapshot.loadHalo(basePath_1, snapshot_ind, halo_ind,'dm')['ParticleIDs']
    
    inter_max = 0
    halo_ind_2 = None
    
    for i in range(len(halo_indices_2)):
        cur_ind = halo_indices_2[i]
        particle_ids_2 = il.snapshot.loadHalo(basePath_2, snapshot_ind, cur_ind,'dm')['ParticleIDs']
        
        inter_cur = len(np.intersect1d(particle_ids_1, particle_ids_2))
        
        if inter_cur > inter_max:
            inter_max = inter_cur
            halo_ind_2 = cur_ind
            
    return halo_ind_2


ind_pairs = []

halo_indices_1 = halo_indices
halo_indices_2 = halo_indices_dark

def progress_bar(cur_val, final_val):
    """ 
    Function to keep track of progress during computations by displaying
    a progress bar

    Parameters:
    cur_val (int/float): current iteration/value calculation is on
    final_val (int/float): final iteration/value that calculation will take
    """

    bar_length = 20
    percent = float(cur_val) / final_val
    arrow = '-' * int(round(percent * bar_length)-1) + '>'
    spaces = ' ' * (bar_length - len(arrow))

    sys.stdout.write("\rProgress: [{0}]"
                     " {1}%".format(arrow + spaces, int(round(percent * 100))))
    sys.stdout.flush()


for i in range(len(halo_indices)):
    
    progress_bar(i, len(halo_indices))
    halo_ind = halo_indices[i]
    
    most_similar_ind = find_most_similar_ind(halo_ind, basePath, halo_indices_2, basePath_dark)
    
    if most_similar_ind != None:
        most_similar_ind_2 = find_most_similar_ind(most_similar_ind, basePath_dark, halo_indices_1, basePath)
        
        if most_similar_ind_2 == halo_ind:
            ind_pairs.append((halo_ind, most_similar_ind))
            halo_indices_1 = np.delete(halo_indices_1, np.argwhere(halo_indices_1==halo_ind))
            halo_indices_2 = np.delete(halo_indices_2, np.argwhere(halo_indices_2==most_similar_ind))

            
        else:
            halo_indices_1 = np.delete(halo_indices_1, np.argwhere(halo_indices_1==halo_ind))
            
    else:
        halo_indices_1 = np.delete(halo_indices_1, np.argwhere(halo_indices_1==halo_ind))
            

Progress: [>                   ] 3%

OSError: Can't read data (file read failed: time = Fri Jun 12 21:19:22 2020
, filename = '../Splashback-Research/TNG-300_3/outputs//snapdir_066/snap_066.1.hdf5', file descriptor = 56, errno = 6, error message = 'Device not configured', buf = 0x7fa795d07000, total read size = 130908, bytes this sub-read = 130908, bytes actually read = 18446744073709551615, offset = 1321549824)